In [59]:
from IPython.display import Image, Math

import pandas as pd
import numpy as np

# Импортируем библиотеки для визуализаци данных
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig()
logger = logging.getLogger("model")
logger.setLevel(logging.INFO)

<br><br>
## TASK 1

In [60]:
# Import data
sales_df = pd.read_parquet("./data/hw/sales.parquet")

# Assign corect data types
sales_df["ds"] = sales_df["ds"].astype(str)

In [61]:
# Get the latest prices for each SKU
prices_df = sales_df \
    .sort_values(["ds"], ascending=False) \
    .groupby(["sku_id"], as_index=False) \
    .head(1) \
    .filter(["sku_id", "price"])

# Get unique combinations of dates and groups (categories)
ds_df = pd.DataFrame({"ds": ["20240101", "20240102", "20240103", "20240104", "20240105", "20240106", "20240107"]})
group_df = sales_df[["sku_id"]].drop_duplicates()

algo_sample = ds_df \
    .join(group_df,
          how="cross") \
    .merge(prices_df,
           how="left",
           left_on="sku_id",
           right_on="sku_id")

# Concatenate with the main DF
sales_df = pd.concat([sales_df, algo_sample])

In [62]:
from typing import List, Dict, Any, Callable, Optional, Callable

# Функция для предсказания базового спроса
def predict_base_demand(
    df: pd.DataFrame,
    W: int,
) -> List[float]:
    preds = df[f"rolling_quantity_w_{W}"]
    return preds

In [63]:
from typing import List, Tuple
# Добавляем новые фичи (окна, сезонные фичи)


# Расчет оконных функций
def calc_rolling_window(
    df: pd.DataFrame,
    window: int,
    col: str,
    lvl: str,
    shift: int,
) -> pd.DataFrame:
    df = df.sort_values(["ds", lvl])
    df["ts"] = pd.to_datetime(df["ds"].astype(str))
    df["col"] = df[col]
    rolling_df = df.set_index("ts")
    rolling_df = (
        rolling_df.groupby([lvl], group_keys=True)["col"]
        # используем shift, так как потом нам нужно предсказывать на N дней вперед:
        # для однородности датасета используем сдвиг на кол-во дат в предсказании
        .apply(lambda x: x.asfreq("1D").rolling(window=window, closed="left", min_periods=0).mean().shift(shift))
        .reset_index()
        .rename(columns={"col": f"rolling_{col}_w_{window}"})
    )
    df = df.merge(rolling_df, how="left", on=[lvl, "ts"])
    df = df.drop(columns=["ts", "col"])
    return df
    
# Функция для преобразования данных
def postprocess_transform(
    df: pd.DataFrame,
    norms: List[Tuple[str, str]],
    roll_cols: List[str],
    windows: List[int],
    dropna_cols: List[str],
    lvl: str,
    shift: int,
):
    # new features
    for window in windows:
        for col in roll_cols:
            logger.info(f"Rolling window={window} days for col `{col}`")
            df = calc_rolling_window(df=df, window=window, col=col, lvl=lvl, shift=shift)

    # normalisation
    # нормализация даст сигнал модели об изменении признаков: если изменилась цена, то к какому изменению спроса это привело?
    for col1, col2 in norms:
        logger.info(f"Normalizing `{col1}` / `{col2}`")
        df[col1] = df[col1] / df[col2]

    # postprocessing
    # для однородности данных удаляем первые даты, по которым собирались окна не по полным данным
    disadvantaged_ds_list = sorted(df["ds"].unique())[:max(shift, max(windows))]
    df = df[~df["ds"].isin(disadvantaged_ds_list)]

    df = df.dropna(subset=dropna_cols)
    df = df.round(2)
    df = df.sort_values(["ds", lvl])
    return df

# Функция для подсчета дуговой эластичности
def calc_elasticity(df: pd.DataFrame, lvl: str) -> pd.DataFrame:
    # сортируем для взятия предыдущих значений
    df = df.sort_values(["ds", lvl])
    # считаем дуговую эластичность
    df["prev_orders_num"] = df.groupby(lvl)["orders_num"].shift(1)
    df["prev_price"] = df.groupby(lvl)["price"].shift(1)
    df["elasticity"] = (
        ((df["orders_num"] - df["prev_orders_num"]) / (df["price"] - df["prev_price"]))
        * ((df["price"] + df["prev_price"]) / (df["orders_num"] + df["prev_orders_num"]))
    )
    return df

# Функция для преобразования данных с параметрами для удобства короткого вызова
def create_features(df: pd.DataFrame, dropna_cols: List[str], lvl: str) -> pd.DataFrame:
    df["day_of_week"] = pd.DatetimeIndex(df["ds"]).day_of_week
    df = calc_elasticity(df=df, lvl=lvl)
    df = postprocess_transform(
        df=df,
        norms=[],
        roll_cols=["orders_num", "elasticity", "price"],
        dropna_cols=dropna_cols,
        windows=[14],
        lvl=lvl,
        shift=7,
    )
    return df

In [64]:
# df = create_features(df=sales_df, dropna_cols=["orders_num", "elasticity", "price"], lvl="sku_id")
# df.head()

In [65]:
# Get rolling window average for ORDERS_NUM and GMV
df_01 = calc_rolling_window(df=sales_df,
                            window=14,
                            col="orders_num",
                            lvl="sku_id",
                            shift=7)

df_01 = calc_rolling_window(df = df_01,
                            window=14,
                            col="gmv",
                            lvl="sku_id",
                            shift=7)

In [66]:
# Get predictions in necessary format for checker
df_01 = df_01 \
    .query(f"(ds >= '20240101') & (ds <= '20240107')") \
    .filter(["sku_id", "ds", "rolling_gmv_w_14", "rolling_orders_num_w_14"]) \
    .rename(columns={"rolling_orders_num_w_14": "orders_num",
                     "rolling_gmv_w_14": "gmv"}) \
    .sort_values(["ds", "sku_id"])

# Save the results
df_01.to_csv("./data/hw/homework_5_1_1.csv", index=False)

<br><br>
## TASK 2

In [67]:
lvl="sku_id"

In [68]:
# Sort the values before shifting
elasticity_df = sales_df \
    .query(f"gmv.notna()") \
    .sort_values(["ds", lvl])

# Shift the values by 1 day
elasticity_df["prev_gmv"] = elasticity_df.groupby(lvl)["gmv"].shift(1)
elasticity_df["prev_orders_num"] = elasticity_df.groupby(lvl)["orders_num"].shift(1)
elasticity_df["prev_price"] = elasticity_df.groupby(lvl)["price"].shift(1)

# Calculate GMV elasticity
elasticity_df["elasticity_gmv"] = (
        ((elasticity_df["gmv"] - elasticity_df["prev_gmv"]) / (elasticity_df["price"] - elasticity_df["prev_price"]))
        * ((elasticity_df["price"] + elasticity_df["prev_price"]) / (elasticity_df["gmv"] + elasticity_df["prev_gmv"]))
    )

# Calculate ORDERS_NUM elasticity
elasticity_df["elasticity_orders_num"] = (
        ((elasticity_df["orders_num"] - elasticity_df["prev_orders_num"]) / (elasticity_df["price"] - elasticity_df["prev_price"]))
        * ((elasticity_df["price"] + elasticity_df["prev_price"]) / (elasticity_df["orders_num"] + elasticity_df["prev_orders_num"]))
    )

# Filter out cases where previous price is the same as current
elasticity_df = elasticity_df \
    .query(f"price != prev_price")

In [69]:
# Filter out cases where elasticity is not within the range
elasticity_gmv = elasticity_df \
    .groupby(["sku_id"], as_index=False) \
    .agg(elasticity_gmv = ("elasticity_gmv", "mean"))
elasticity_gmv["elasticity_gmv"] = elasticity_gmv["elasticity_gmv"].clip(lower=-3.00, upper=-0.01)

elasticity_orders_num = elasticity_df \
    .groupby(["sku_id"], as_index=False) \
    .agg(elasticity_orders_num = ("elasticity_orders_num", "mean"))
elasticity_orders_num["elasticity_orders_num"] = elasticity_orders_num["elasticity_orders_num"].clip(lower=-3.00, upper=-0.01)

elasticity_df = elasticity_df \
    .filter(["sku_id"]) \
    .drop_duplicates() \
    .merge(elasticity_gmv,
           how="left",
           left_on="sku_id",
           right_on="sku_id") \
    .merge(elasticity_orders_num,
           how="left",
           left_on="sku_id",
           right_on="sku_id")

# Save the results
elasticity_df.to_csv("./data/hw/homework_5_1_2.csv", index=False)

<br><br>
## TASK 3

In [70]:
# 1 - взял за основу sales.parquet
# 2 - для периода предсказаний (20240101 - 20240107) взял последнюю доступную по дате цену на уровне "sku_id"
# 3 - подсчитал rolling window average по цене со следующими параметрами: lvl="sku_id", window=14, shift=7
# 4 - сгенерировал сетку из "ds" (20240101 - 20240107) и "discount", заджойнил данные выше
# 5 - заджойнил базовый спрос ("gmv", "orders_num") из homework_5_1_1.csv
# 6 - заджойнил эластичности из homework_5_1_2.csv
# 7 - посчитал предсказания "gmv" и "orders_num"
# 8 - посчитал "margin"
# 9 - сохранил результат и отправил в LMS

In [71]:
rolling_price_df = calc_rolling_window(df=sales_df,
                                       window=14,
                                       col="price",
                                       lvl="sku_id",
                                       shift=7)

In [72]:
prediction_df = rolling_price_df \
    .query(f"'20240101' <= ds <= '20240107'") \
    .filter(["sku_id", "ds", "price", "rolling_price_w_14"]) \
    .merge(df_01,
           how="left",
           on=["sku_id", "ds"]) \
    .merge(elasticity_df,
           how="left",
           on="sku_id")

In [73]:
# Get unique combinations of dates and groups (categories)
ds_df = pd.DataFrame({"ds": ["20240101", "20240102", "20240103", "20240104", "20240105", "20240106", "20240107"]})
group_df = pd.DataFrame({"discount": [-0.1, -0.08, -0.06, -0.04, -0.02, 0.0, 0.02, 0.04, 0.06, 0.08, 0.1]})

discount_df = ds_df \
    .join(group_df,
          how="cross") \
    .reset_index(drop=True)

In [74]:
prediction_df = discount_df \
    .merge(prediction_df,
           how="left",
           on="ds")

In [75]:
prediction_df = prediction_df \
    .assign(price = lambda x: x["price"] + x["price"] * x["discount"]) \
    .assign(orders_num = lambda x: x["orders_num"] * np.power(x["price"] / x["rolling_price_w_14"], x["elasticity_orders_num"]),
            gmv = lambda x: x["gmv"] * np.power(x["price"] / x["rolling_price_w_14"], x["elasticity_gmv"])) \
    .assign(margin = lambda x: x["gmv"] * 0.1 + x["gmv"] * x["discount"]) \
    .filter(["sku_id", "ds", "discount", "orders_num", "gmv", "margin"])

# Save the results
prediction_df.to_csv("./data/hw/homework_5_2.csv", index=False)

<br><br>
## TASK 4

In [76]:
import logging
from typing import List, Dict, Any

import numpy as np
import pandas as pd

logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


def log_uplifts(
    constraints: Dict[str, float],
    maximized_column: str,
    optimal_statistics: Dict[str, float],
) -> None:
    """
    Функция для логирования значений метрик и их аплифтов (улучшений).

    :param constraints: Словарь ограничений для метрик.
    :param maximized_column: Название столбца, который подлежит максимизации.
    :param optimal_statistics: Словарь с оптимальными статистическими данными.
    """
    # Логируем значение метрики, которую мы максимизируем
    logger.info(f"Metric: {maximized_column}", extra={"value": optimal_statistics.get(maximized_column)})

    # Проходим по всем метрикам и их ограничениям
    for metric, constraint in constraints.items():
        optimal_value = optimal_statistics.get(metric)
        if optimal_value is None:
            raise ValueError(f"`{metric}` has not been counted")
        # Логируем информацию по каждой метрике, включая аплифты
        log_dict = {
            "constraint value": round(constraint, 3),
            "optimal value": round(optimal_value, 3),
            "uplift (abs)": round(optimal_value - constraint, 3),
            "uplift (pct)": round(optimal_value * 100 / constraint - 100, 3),
        }
        logger.info(f"Metric: {metric}")
        for key, value in log_dict.items():
            logger.info(f"{key}= {value}")


def apply_constraints(
    df: pd.DataFrame,
    constraints: Dict[str, float],
) -> pd.DataFrame:
    """
    Фильтруем датасет по заданным ограничениям.

    :param df: DataFrame с данными для фильтрации.
    :param constraints: Словарь ограничений для каждой метрики.
    :return: Отфильтрованный DataFrame.
    """
    # Применяем ограничения к датафрейму, фильтруя строки
    for metric, constraint in constraints.items():
        df = df[df[metric] >= constraint]
    return df


def calculate_cum_lambda_metrics(
    df: pd.DataFrame,
    agg_columns: List[str],
    maximized_column: str,
) -> pd.DataFrame:
    """
    Считаем агрегированные значения метрик для каждой комбинации лямбда-значений.

    :param df: DataFrame с данными.
    :param agg_columns: Список столбцов для агрегации.
    :param maximized_column: Столбец, который максимизируется.
    :return: Агрегированный DataFrame.
    """
    # Группируем данные по комбинации лямбда-значений и агрегируем указанные столбцы
    df = df.groupby("lambda_combination").agg({column: "sum" for column in agg_columns})
    df = df.reset_index()
    return df


def choose_optimal_values(
    metric_lambda_map: Dict[str, float],
    df: pd.DataFrame,
    levels: List[str],
    price_column: str,
    maximized_column: str,
) -> pd.DataFrame:
    """
    Находим оптимальные цены / наценки для каждого уровня для lambda_value
    """
    # Считаем лагранжианы при lambda_value
    df["lagrangian"] = df[maximized_column]
    lambda_combination_name = ""
    for metric, metric_lambda in metric_lambda_map.items():
        df["lagrangian"] += df[metric] * metric_lambda
        lambda_combination_name += f"{metric}={metric_lambda}_"
    # Находим максимальный лагранжиан для каждого уровня
    optimal_df = df.groupby(levels).agg({"lagrangian": "max"})
    df = df.merge(optimal_df, on=levels + ["lagrangian"], how="inner")
    # Добавляем колонку с lambda_value для запоминания
    df["lambda_combination"] = lambda_combination_name.strip("_")
    # Удаляем дубликаты (например, оставляем минимальные цены / наценки из оптимальных),
    # так как возможны одни и те же значения метрик для разных цен / наценок
    # => одинаковые лагранжианы, а нам нужно выбрать одно значение для каждого уровня
    df = df.sort_values(price_column)
    df = df.drop_duplicates(subset=levels)
    return df


def get_metric_lambda_maps(lambda_config: Dict[str, Any]) -> List[Dict[str, float]]:
    # Получаем список значений для каждого ключа
    lambda_lists = list(lambda_config.values())
    # Используем meshgrid для генерации всех комбинаций параметров
    lambda_mesh = np.meshgrid(*lambda_lists)
    # Преобразование в массив и решейпинг
    lambda_vars = np.stack(lambda_mesh, axis=-1).reshape(-1, len(lambda_config))
    # Создаем список словарей
    metric_lambda_maps = [
        dict(zip(lambda_config.keys(), combination)) for combination in lambda_vars
    ]
    return metric_lambda_maps


def calculate_lagrangians(
    df: pd.DataFrame,
    lambda_config: Dict[str, Any],
    levels: List[str],
    price_column: str,
    maximized_column: str,
) -> pd.DataFrame:
    """
    Для каждого значения lambda находим оптимальные цены / наценки для каждого уровня
    """
    lambda_dfs = []
    metric_lambda_maps = get_metric_lambda_maps(lambda_config=lambda_config)
    logger.info(
        f"Start calculating lagrangians, {len(metric_lambda_maps)} lambda combinations"
    )
    for metric_lambda_map in metric_lambda_maps:
        lambda_df = choose_optimal_values(
            metric_lambda_map=metric_lambda_map,
            df=df,
            levels=levels,
            price_column=price_column,
            maximized_column=maximized_column,
        )
        lambda_dfs.append(lambda_df)
    df = pd.concat(lambda_dfs)
    df = df.reset_index(drop=True)
    logger.info(f"Ended calculating lagrangians")
    return df


# Общая функция для оптимизации
def optimize(
    df: pd.DataFrame,
    lambda_config: Dict[str, Any],
    maximized_column: str,
    constraints: Dict[str, float],
    levels: List[str],
    price_column: str,
) -> pd.DataFrame:
    logger.info("Start choosing optimal prices")
    lambda_df = calculate_lagrangians(
        df=df,
        lambda_config=lambda_config,
        levels=levels,
        price_column=price_column,
        maximized_column=maximized_column,
    )
    statistics_df = calculate_cum_lambda_metrics(
        df=lambda_df,
        agg_columns=[maximized_column] + list(constraints.keys()),
        maximized_column=maximized_column,
    )
    statistics_df = statistics_df.sort_values(maximized_column, ascending=False)
    logger.info(f"\n{statistics_df.head()}")
    statistics_df = apply_constraints(df=statistics_df, constraints=constraints)
    logger.info(f"\n{statistics_df.head()}")
    best_lambda = statistics_df["lambda_combination"].tolist()[0]
    optimal_statistics = statistics_df[
        statistics_df["lambda_combination"] == best_lambda
    ].to_dict(orient="records")[0]
    optimal_df = lambda_df[lambda_df["lambda_combination"] == best_lambda]
    log_uplifts(
        constraints=constraints,
        maximized_column=maximized_column,
        optimal_statistics=optimal_statistics,
    )
    logger.info("Ended choosing optimal prices")
    return optimal_df

In [77]:
prediction_df

,sku_id,ds,discount,orders_num,gmv,margin
0,1,20240101,-0.1,2363.433516,3.189778e+04,0.000000
1,3,20240101,-0.1,1241.987842,1.494497e+04,0.000000
2,4,20240101,-0.1,3924.782603,2.281482e+05,0.000000
3,7,20240101,-0.1,5218.345346,1.162171e+06,0.000000
4,8,20240101,-0.1,5821.836268,6.155718e+05,0.000000
...,...,...,...,...,...,...
16473,393,20240107,0.1,169.915750,1.733853e+05,34677.065743
16474,398,20240107,0.1,301.157897,8.112007e+03,1622.401463
16475,399,20240107,0.1,59.358236,2.302252e+03,460.450319
16476,400,20240107,0.1,33.588556,1.849073e+03,369.814641


In [78]:
# Считаем ограничения
control_margin = prediction_df[prediction_df["discount"] == 0]["margin"].sum()
control_gmv = prediction_df[prediction_df["discount"] == 0]["gmv"].sum()

In [79]:
np.arange(0.0, 1.1, 0.1).tolist()

[0.0,
 0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6000000000000001,
 0.7000000000000001,
 0.8,
 0.9,
 1.0]

In [80]:
optimal_df = optimize(
    df=prediction_df,
    # перебираем разные lambda для выручки
    lambda_config={
        "margin": np.arange(0.0, 1.1, 0.1).tolist(),
        "gmv": np.arange(0.0, 1.1, 0.1).tolist()
    },
    # указываем, что хотим максимизировать
    maximized_column="orders_num",
    # указываем ограничения
    constraints={
        "margin": control_margin,
        "gmv": control_gmv
    },
    levels=["sku_id", "ds"],
    price_column="discount",
)

INFO:__main__:Start choosing optimal prices
INFO:__main__:Start calculating lagrangians, 121 lambda combinations
INFO:__main__:Ended calculating lagrangians
INFO:__main__:
                   lambda_combination    orders_num  margin           gmv
0                  margin=0.0_gmv=0.0  3.357924e+06     0.0  1.976511e+08
6   margin=0.0_gmv=0.6000000000000001  3.357924e+06     0.0  1.976511e+08
10                 margin=0.0_gmv=1.0  3.357924e+06     0.0  1.976511e+08
9                  margin=0.0_gmv=0.9  3.357924e+06     0.0  1.976511e+08
8                  margin=0.0_gmv=0.8  3.357924e+06     0.0  1.976511e+08
INFO:__main__:
                   lambda_combination    orders_num        margin  \
21                 margin=0.1_gmv=1.0  3.309405e+06  3.426072e+07   
19                 margin=0.1_gmv=0.8  3.308614e+06  3.426947e+07   
20                 margin=0.1_gmv=0.9  3.308614e+06  3.426947e+07   
16                 margin=0.1_gmv=0.5  3.308614e+06  3.426947e+07   
17  margin=0.1_gmv=0.600

In [81]:
# Save the results
optimal_df \
    .filter(["sku_id", "ds", "discount"]) \
    .to_csv("./data/hw/homework_5_3.csv", index=False)